In [2]:
#-*- coding: UTF-8 -*-
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import f1_score,accuracy_score,roc_auc_score,log_loss, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer
import gc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
pd.options.display.max_columns=50
pd.options.display.max_rows=20

In [2]:
def run_lgb_cv(train_X,train_Y,lgb_params,feature_name=None,split=5,seed=6666,cat_list=None, threshold = 0.5):
    val_results = []
    models_list = []
    best_iterations = []
    train_pred = np.zeros(train_X.shape[0])
    

    # recall 和precision 的score
    def lgb_self_score(y_pred, data):
        y_true = data.get_label()  #lgb_data获取label
        # y_pred = np.round(y_pred) # 默认精度为0，四舍五入
        y_pred = np.where(y_pred>=threshold,1,0)
        return [('accuracy_score', accuracy_score(y_true, y_pred), True),('precision_score', precision_score(y_true, y_pred), True),('recall_score', recall_score(y_true, y_pred), True)]

        

    seeds=range(seed,seed+split)
    feature_importance_df = pd.DataFrame()
    
    if feature_name == None:
        feature_name = [col for col in train_X.columns if col!='index' and col!='id' and col!='isNew']
    print('Using features:',feature_name)
    
    
    # 分类器GroupKFold
    gkf = GroupKFold(n_splits=split)
    index = 0
    for train_index, test_index in gkf.split(train_X, train_Y, groups=train_X['ID']):
        
    # for index, (train_index, test_index) in enumerate(train_val_spliter.split(train_X, train_Y)):
        print('fold:',index+1)
        lgb_params['random_state'] = seeds[index]
        index += 1 
        
        val_result = []
        #划分训练集及标签，验证集及标签
        train_x, val_x, train_y, val_y = train_X[feature_name].iloc[train_index], train_X[feature_name].iloc[test_index], train_Y.iloc[train_index], train_Y.iloc[test_index]
        
        # 标签0与标签1的比
        neg_pos_ratio = np.log(len(train_y[train_y==0])/len(train_y[train_y==1]))
        #neg_pos_ratio = 1
        print('neg_pos_ratio:',neg_pos_ratio)
        
        # 使用lgb的必须操作：转化为lgb的dataset
        train_data = lgb.Dataset(train_x,label=train_y)
        val_data = lgb.Dataset(val_x,label=val_y)
        # 种类特征的相关处理
        if cat_list == None:
            cat_list = ""
        #lgb_model = lgb.LGBMClassifier(**lgb_params)
        # 进行训练
        metric_dict = {}
        lgb_model=lgb.train(lgb_params,train_data,valid_sets=[val_data],verbose_eval=lgb_params['verbose'],
                            callbacks = [lgb.record_evaluation(metric_dict)], feval=lgb_self_score)
        #lgb_model.fit(train_x,train_y,eval_set=(val_x,val_y))
        gc.collect() # 清内存
        
        # 把验证集预测结果按照test_index加到predict矩阵中，这样数折验证后刚好填满train_pred矩阵
        train_pred[test_index] += lgb_model.predict(val_x[feature_name])
        
        # 生成有feature和importance的特征重要度矩阵
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = pd.Series(feature_name)
        fold_importance_df["importance"] = pd.Series(lgb_model.feature_importance(importance_type='gain'))
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        # 计算分数
        val_result.append(accuracy_score(val_y, np.where(train_pred[test_index]>=threshold,1,0)))
        print('accuracy_score: ',val_result[-1])
        val_result.append(precision_score(val_y, np.where(train_pred[test_index]>=threshold,1,0)))
        print('precision_score: ',val_result[-1])
        val_result.append(recall_score(val_y, np.where(train_pred[test_index]>=threshold,1,0)))
        print('recall_score: ',val_result[-1])
        
        # 记录数据
        best_iterations.append(lgb_model.current_iteration())
        models_list.append(lgb_model)
        val_results.append(val_result)
    

    
    # 打印特征重要度
    # best_features返回模型中平均重要度的降序排列
    best_features = (feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False))
    best_features['feature'] = best_features.index  # 因为这样操作后index是特征名，需要修改
    best_features.reset_index(drop=True)
    plt.figure(figsize=(14,20))
    sns.barplot(x="importance", y="feature", data=best_features)
    plt.title('LightGBM Features (averaged over folds)')
    plt.tight_layout()

    val_results = np.array(val_results)
    print('cv completed')
    print('mean best iteration: ',np.mean(best_iterations))
    print('std best iteration: ',np.std(best_iterations))   # 标准差
    print('-'*40)
    
    # print('oof AUC: ',roc_auc_score(train_Y,train_pred))
    # print('mean AUC: ',np.mean(val_results[:,0]))
    # print('std AUC: ',np.std(val_results[:,0]))
    # print('mean f1_score: ',np.mean(val_results[:,1]))
    # print('std f1_score: ',np.std(val_results[:,1]))
    # print('oof f1_score: ',f1_score(train_Y,train_pred))
    train_pred = np.where(train_pred>=threshold,1,0)
    # 各折平均分数
    # print('mean accuracy_score: ',np.mean(val_results[:,0]))
    # print('mean precision_score: ',np.mean(val_results[:,1]))
    # print('mean recall_score: ',np.mean(val_results[:,2]))
    # 各折分数标准差，标准差越低越好
    dic = [{'平均迭代轮数 ':np.mean(best_iterations), '精确率方差':np.std(val_results[:,1]),'召回率方差':np.std(val_results[:,2]),
            '准确度率分数':accuracy_score(train_Y,train_pred),'精确率分数':precision_score(train_Y,train_pred),
            '召回率分数':recall_score(train_Y, train_pred),
            '分数综合':(accuracy_score(train_Y,train_pred)+precision_score(train_Y,train_pred)+recall_score(train_Y, train_pred))/3},
           {'平均迭代轮数 ':np.mean(best_iterations), '精确率方差':np.std(val_results[:,1]),'召回率方差':np.std(val_results[:,2]),
            '准确度率分数':accuracy_score(train_Y,train_pred),'精确率分数':precision_score(train_Y,train_pred),
            '召回率分数':recall_score(train_Y, train_pred),
            '分数综合':(accuracy_score(train_Y,train_pred)+precision_score(train_Y,train_pred)+recall_score(train_Y, train_pred))/3}]
    score_result = pd.DataFrame(dic,index = [0,1])
    #print('std accuracy_score: ',np.std(val_results[:,0]))
    #print('std precision_score: ',np.std(val_results[:,1]))
    #print('std recall_score: ',np.std(val_results[:,2]))
    # 最终分数，分数越高越好
    #print('oof accuracy_score：', accuracy_score(train_Y,train_pred))   # 准确率 （TP+TN）/（TP+TN+FN+FP）
    #print('oof precision_score：', precision_score(train_Y,train_pred))  # 精确率  TP  / (TP+ FP)  预测为正的样本中有多少为真
    #print('oof recall_score：', recall_score(train_Y, train_pred))    # 召回率  TP / (TP + FN)   结果为正的样本中有多少被预测正确了
    
    return train_pred,models_list,pd.DataFrame(val_results,columns=['accuracy_score','precision_score', 'recall_score']),best_features.sort_values(by="importance",ascending=False),score_result

In [28]:
# 训练集读取，45146*30，其中有标签的15215*30
train_data = pd.read_csv('base_train_sum.csv', encoding = 'gbk')
train_data = pd.merge(train_data,pd.read_csv('knowledge_train_sum.csv', encoding = 'gbk'),how='left',on='ID')
train_data = pd.merge(train_data,pd.read_csv('money_report_train_sum.csv', encoding = 'gbk'),how='left',on='ID')
train_data = pd.merge(train_data,pd.read_csv('year_report_train_sum.csv', encoding = 'gbk'),how='left',on=['ID','year']) # 这样就能一起合并
train_data = train_data[~train_data['flag'].isnull()].reset_index(drop=True)

# 验证集读取，96250*31，其中有标签的91732*30（注：比训练集多了一个控制人ID，官方回复测试集中不会加入这个特征，所以选择删除）
valid_data = pd.read_csv('base_verify1.csv', encoding = 'gbk')
valid_data = pd.merge(valid_data,pd.read_csv('paient_information_verify1.csv', encoding = 'gbk'),how='left',on='ID')
valid_data = pd.merge(valid_data,pd.read_csv('money_information_verify1.csv', encoding = 'gbk'),how='left',on='ID')
valid_data = pd.merge(valid_data,pd.read_csv('year_report_verify1.csv', encoding = 'gbk'),how='left',on=['ID','year'])
del valid_data['控制人ID']

# 合并为一个数据集，137796*30
data = pd.concat([train_data,valid_data],axis=0,sort=False,ignore_index=True)
data.columns = ['ID','Registration Time', 'Registered Capital', 'Industry', 'Region', 'Business Type', 'Controlling Type', 'Controlling Shareholding','flag', 'Patent', 'Trademark', 'Copyright', 'year', 'Debt financing line', 'Debt financing cost', 'Equity financing line', 'Equity financing cost', 'Internal financing and trade financing line', 'Internal financing and trade financing cost', 'Project financing and Policy financing quota ',' Project financing and policy financing costs', 'Number of employees','Total assets', 'Total liabilities',' Total operating income ',' Main operating income ',' Total profit ' , 'Net Profit', 'Total Taxes', 'Total Owners Equity']

# 去flag=null的值，现在data:106947*30
data = data[~data['flag'].isnull()].reset_index(drop=True)
# 打乱数据
# data = data.sample(frac=1).reset_index(drop=True)
# 去重
data = data.drop_duplicates(subset=None,keep='first',inplace=False)
# 所有nan值换0,不要使用，降低效果
# data = data.fillna(0)

#labelencode
#['Industry', 'Region', 'Business Type', 'Controlling Type']
#['行业','区域','企业类型','控制人类型']
for col in ['Industry', 'Region', 'Business Type', 'Controlling Type']:
    lbl = LabelEncoder()
    data[col] = lbl.fit_transform(data[col].astype(str))

In [29]:
# 特征工程 ——自身特征
data['Total Taxes Square'] = data['Total Taxes'] **2
data['log Total Taxes'] = np.log(data['Total Taxes'])
data['log Net Profit'] = np.log(data['Net Profit'])
                                
# qcut等频分箱，cut为等距分箱
cats = pd.qcut(data['Total Taxes'], 6, duplicates = 'drop')
data['Total Taxes qcut_branch6'] = lbl.fit_transform(cats.astype(str))
cats = pd.qcut(data['Net Profit'], 8, duplicates = 'drop')
data['Net Profit qcut_branch8'] = lbl.fit_transform(cats.astype(str))

d:\tools\python\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
d:\tools\python\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [30]:
#特征工程2
datag = data.groupby('ID')
lst = list(set(data['ID']))
Total_Taxes_incre = []         
Total_Taxes_incre_mean = []
Net_profit_has_0 = []
Total_Taxes_has_0 = []
for id in lst:
    group = datag.get_group(id)
    taxes = list(group['Total Taxes'])
    nets = list(group['Net Profit'])
    
    # 判断三年利润为0/有1~2年为0/都存在值
    if sum(nets)==0:  
        Net_profit_has_0.extend([0]*len(nets))
    elif(0 in nets):
        Net_profit_has_0.extend([1]*len(nets)) 
    else:
        Net_profit_has_0.extend([2]*len(nets)) 
    
    # 判断三年税收为0/有1~2年为0/都存在值
    if sum(taxes)==0:
        Total_Taxes_has_0.extend([0]*len(taxes))
    elif(0 in taxes):
        Total_Taxes_has_0.extend([1]*len(taxes)) 
    else:
        Total_Taxes_has_0.extend([2]*len(taxes)) 
    
    # 计算总税收增长率
    if 0 in taxes or len(taxes)<3:
        Total_Taxes_incre_mean.extend([0]*len(taxes))
    else:
        tax_mean = (taxes[1]/taxes[0] + taxes[2]/taxes[1] )/2
        Total_Taxes_incre_mean.extend([tax_mean]*len(taxes))
        
data['Net_profit_has_0'] = Net_profit_has_0
data['Total_Taxes_has_0'] = Total_Taxes_has_0  
data['Total_Taxes_incre_mean'] = Total_Taxes_incre_mean


In [31]:
# 特征工程3
data['asset_is_smaller_than_liabilities'] = data['Total assets'] < data['Total liabilities']#资不抵债
data['year*Net Profit'] = data['year'] * data['Net Profit']#搜索特征
data['Industry*Total Taxes'] = data['Industry'] * data['Total Taxes'] # 搜索特征
data['Asset liability ratio'] = data['Total assets']/data['Total liabilities'] #资产负债比率

In [32]:

data1 = data.fillna(0)
data1 = data1.replace([np.inf, -np.inf],0) #替换正负inf为NA
del data1['ID']  #ID和Flag对聚类影响极大，需要先行删除
del data1['flag']
estimator = KMeans(n_clusters=6, n_jobs = 8) #构造聚类器
estimator.fit(data1)
label_pred = estimator.labels_#最终聚类类别  

data['Kmeans_label'] = label_pred  # 聚类结果作为一维特征

d:\tools\python\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


In [33]:
data

,ID,Registration Time,Registered Capital,Industry,Region,Business Type,Controlling Type,Controlling Shareholding,flag,Patent,Trademark,Copyright,year,Debt financing line,Debt financing cost,Equity financing line,Equity financing cost,Internal financing and trade financing line,Internal financing and trade financing cost,Project financing and Policy financing quota,Project financing and policy financing costs,Number of employees,Total assets,Total liabilities,Total operating income,Main operating income,Total profit,Net Profit,Total Taxes,Total Owners Equity,Total Taxes Square,log Total Taxes,log Net Profit,Total Taxes qcut_branch6,Net Profit qcut_branch8,Net_profit_has_0,Total_Taxes_has_0,Total_Taxes_incre_mean,asset_is_smaller_than_liabilities,year*Net Profit,Industry*Total Taxes,Asset liability ratio,Kmeans_label
0,28,2007.0,2050.0,1,7,1,1,NaN,1.0,0.0,1.0,1.0,2015.0,0.0,0.00,0.0,0.000,21648.0,1298.880,0.0,0.000,794.0,16400.0,28700.0,72160.0,28864.0,7216.0,-7216.0,0.0,-12300.0,0.000000e+00,-inf,NaN,0,0,2,0,0.0,True,-14540240.0,0.0,0.571429,0
1,28,2007.0,2050.0,1,7,1,1,NaN,1.0,0.0,1.0,1.0,2016.0,0.0,0.00,34686.0,1387.440,0.0,0.000,0.0,0.000,396.0,73800.0,71750.0,346860.0,173430.0,173430.0,-34686.0,0.0,2050.0,0.000000e+00,-inf,NaN,0,2,2,0,0.0,False,-69926976.0,0.0,1.028571,0
2,28,2007.0,2050.0,1,7,1,1,NaN,1.0,0.0,1.0,1.0,2017.0,0.0,0.00,3444.0,137.760,0.0,0.000,0.0,0.000,393.0,82000.0,159900.0,172200.0,103320.0,17220.0,-17220.0,0.0,-77900.0,0.000000e+00,-inf,NaN,0,2,2,0,0.0,True,-34732740.0,0.0,0.512821,0
3,230,2008.0,3360.0,4,2,1,1,1.00,1.0,0.0,0.0,0.0,2015.0,0.0,0.00,0.0,0.000,0.0,0.000,470.4,28.224,485.0,23520.0,10080.0,115248.0,57624.0,57624.0,-11524.8,0.0,13440.0,0.000000e+00,-inf,NaN,0,0,2,0,0.0,False,-23222472.0,0.0,2.333333,0
4,230,2008.0,3360.0,4,2,1,1,1.00,1.0,0.0,0.0,0.0,2016.0,0.0,0.00,0.0,0.000,46771.2,2806.272,0.0,0.000,365.0,53760.0,50400.0,155904.0,124723.2,46771.2,-15590.4,0.0,3360.0,0.000000e+00,-inf,NaN,0,0,2,0,0.0,False,-31430246.4,0.0,1.066667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106942,5999998,2014.0,4510.0,4,6,4,2,0.64,0.0,0.0,0.0,0.0,2017.0,0.0,0.00,0.0,0.000,18265.5,1095.930,0.0,0.000,841.0,67650.0,63140.0,60885.0,30442.5,30442.5,-6088.5,0.0,4510.0,0.000000e+00,-inf,NaN,0,0,2,0,0.0,False,-12280504.5,0.0,1.071429,0
106943,5999999,2014.0,9130.0,1,7,4,2,0.80,0.0,1.0,1.0,1.0,2015.0,0.0,0.00,6025.8,241.032,0.0,0.000,0.0,0.000,885.0,27390.0,9130.0,60258.0,42180.6,30129.0,6025.8,24103.2,18260.0,5.809643e+08,10.090100,8.703806,1,6,2,0,0.0,False,12141987.0,24103.2,3.000000,0
106944,5999999,2014.0,9130.0,1,7,4,2,0.80,0.0,1.0,1.0,1.0,2016.0,7304.0,584.32,0.0,0.000,0.0,0.000,0.0,0.000,933.0,73040.0,95865.0,51128.0,25564.0,5112.8,0.0,10225.6,-22825.0,1.045629e+08,9.232650,-inf,1,3,2,0,0.0,True,0.0,10225.6,0.761905,0
106945,5999999,2014.0,9130.0,1,7,4,2,0.80,0.0,1.0,1.0,1.0,2017.0,0.0,0.00,0.0,0.000,0.0,0.000,821.7,49.302,46.0,82170.0,73040.0,16434.0,11503.8,8217.0,6573.6,9860.4,9130.0,9.722749e+07,9.196282,8.790817,1,6,2,0,0.0,False,13258951.2,9860.4,1.125000,0


In [34]:
# 获取特征名
feature_name = [col for col in data.columns if col not in ['ID','flag']]
print(feature_name)

['Registration Time', 'Registered Capital', 'Industry', 'Region', 'Business Type', 'Controlling Type', 'Controlling Shareholding', 'Patent', 'Trademark', 'Copyright', 'year', 'Debt financing line', 'Debt financing cost', 'Equity financing line', 'Equity financing cost', 'Internal financing and trade financing line', 'Internal financing and trade financing cost', 'Project financing and Policy financing quota ', ' Project financing and policy financing costs', 'Number of employees', 'Total assets', 'Total liabilities', ' Total operating income ', ' Main operating income ', ' Total profit ', 'Net Profit', 'Total Taxes', 'Total Owners Equity', 'Total Taxes Square', 'log Total Taxes', 'log Net Profit', 'Total Taxes qcut_branch6', 'Net Profit qcut_branch8', 'Net_profit_has_0', 'Total_Taxes_has_0', 'Total_Taxes_incre_mean', 'asset_is_smaller_than_liabilities', 'year*Net Profit', 'Industry*Total Taxes', 'Asset liability ratio', 'Kmeans_label']


In [64]:
# recall 和precision 的score
def lgb_self_score(y_pred, data):
    y_true = data.get_label()  #lgb_data获取label
    # y_pred = np.round(y_pred) # 默认精度为0，四舍五入
    y_pred = np.where(y_pred>=0.5,1,0)
    return [('accuracy_score', accuracy_score(y_true, y_pred)),('precision_score', precision_score(y_true, y_pred)),('recall_score', recall_score(y_true, y_pred))]

        
# 设置参数
param = {'max_depth': 3,
         'eta': 1,
         'objective': 'binary:logistic',
} 
train_X = data
train_Y = data['flag'].astype(int)
train_pred = np.zeros(train_X.shape[0])
# 分类器GroupKFold
gkf = GroupKFold(n_splits=5)
bst_lst = []
for train_index, test_index in gkf.split(train_X, train_Y, groups=train_X['ID']):
    val_result = []
    #划分训练集及标签，验证集及标签
    train_x, val_x, train_y, val_y = train_X[feature_name].iloc[train_index], train_X[feature_name].iloc[test_index], train_Y.iloc[train_index], train_Y.iloc[test_index]
    data_train = xgb.DMatrix(train_x,label=train_y)
    data_test = xgb.DMatrix(val_x,label=val_y)
    
    #bst = xgb.XGBClassifier(**param)
    #bst.fit(train_x, train_y, eval_set=[(train_x, train_y), (val_x, val_y)],
    #                early_stopping_rounds = 100,
    #               feval = lgb_self_score)  
    eval_result = {}
    bst = xgb.train(param, data_train, evals=[(data_train,'train'),(data_test,'vaild')], evals_result = eval_result,
                    num_boost_round = 1000,verbose_eval=20,early_stopping_rounds=100, feval = lgb_self_score, maximize = True)
    bst_lst.append(bst)
    # 把验证集预测结果按照test_index加到predict矩阵中，这样数折验证后刚好填满train_pred矩阵
    #train_pred[test_index] += bst.predict(val_x[feature_name])

[0]	train-error:0.10358	vaild-error:0.10224	train-accuracy_score:0.89642	train-precision_score:0.79684	train-recall_score:0.98818	vaild-accuracy_score:0.89776	vaild-precision_score:0.79965	vaild-recall_score:0.98691
Multiple eval metrics have been passed: 'vaild-recall_score' will be used for early stopping.

Will train until vaild-recall_score hasn't improved in 100 rounds.
[20]	train-error:0.10025	vaild-error:0.10201	train-accuracy_score:0.89975	train-precision_score:0.80400	train-recall_score:0.98479	vaild-accuracy_score:0.89799	vaild-precision_score:0.80320	vaild-recall_score:0.98048
[40]	train-error:0.09812	vaild-error:0.10379	train-accuracy_score:0.90188	train-precision_score:0.81099	train-recall_score:0.97810	vaild-accuracy_score:0.89621	vaild-precision_score:0.80636	vaild-recall_score:0.96821
[60]	train-error:0.09570	vaild-error:0.10496	train-accuracy_score:0.90430	train-precision_score:0.81688	train-recall_score:0.97482	vaild-accuracy_score:0.89504	vaild-precision_score:0.8081

Stopping. Best iteration:
[4]	train-error:0.10158	vaild-error:0.10384	train-accuracy_score:0.89842	train-precision_score:0.79996	train-recall_score:0.98854	vaild-accuracy_score:0.89616	vaild-precision_score:0.79558	vaild-recall_score:0.98988



In [90]:
eval_result['vaild']['accuracy_score'][4]

0.896157

In [86]:
eval_result['vaild']['precision_score'][4]

0.795579

In [87]:
eval_result['vaild']['recall_score'][4]

0.989879

In [77]:
dic = {'a':123}

In [79]:
dic['a']

123